# **DATA PREPARATION & EXPLORATION**

In [98]:
#Mount Google Drive to Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [99]:
#Change Directory
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Tsel_DSA/01_Python_Basic/')
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/Tsel_DSA/01_Python_Basic'

In [0]:
#import packages
import numpy as np
import pandas as pd

In [0]:
#Load file
google = pd.read_csv("googleplaystore.csv", encoding='latin1')

In [102]:
#Check dimension
google.shape

(10841, 13)

In [103]:
#Check Content
google.head(3)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite â FREE Live Cool Themes, Hid...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up


In [104]:
#Show Data Profile
Data_dict = pd.DataFrame(google.dtypes)
Data_dict = Data_dict.rename(columns={0:'Data_Type'})
Data_dict['Missing_Value'] = google.isnull().sum()
Data_dict['Unique_Value'] = google.nunique()
Data_dict['Count'] = google.count()
Data_dict

,Data_Type,Missing_Value,Unique_Value,Count
App,object,0,9660,10841
Category,object,0,34,10841
Rating,float64,1474,40,9367
Reviews,object,0,6002,10841
Size,object,0,462,10841
Installs,object,0,22,10841
Type,object,1,3,10840
Price,object,0,93,10841
Content Rating,object,1,6,10840
Genres,object,0,120,10841


### **Standardize Column Name**

In [0]:
#Replace whitespace in column name with underscore
google.rename(columns= lambda x : x.replace(' ','_') if ' ' in x else x, inplace=True)

In [106]:
#Check column name
google.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content_Rating', 'Genres', 'Last_Updated', 'Current_Ver',
       'Android_Ver'],
      dtype='object')

## **Handling Numeric Column Data Type**

### **Reviews, Size, Installs and Price suppose to be numeric**


* **Convert Reviews data type to numeric**

In [107]:
#Check non numeric values in Reviews
google.Reviews[pd.to_numeric(google.Reviews, errors='coerce').isna()].value_counts()

3.0M    1
Name: Reviews, dtype: int64

In [108]:
#Check the coresponding row
google[google.Reviews=='3.0M']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Current_Ver,Android_Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


  There is row **10472** that column 2 to 13 shifted to the left 1 period

In [0]:
#Shifted row 10472 to right
google.iloc[10472,1:]=google.iloc[10472,1:].shift(1)

In [110]:
#Check the updated row
google[google.App=='Life Made WI-Fi Touchscreen Photo Frame']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Current_Ver,Android_Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,NaN,1.9,19,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up


In [111]:
#Convert Reviews to Numeric
google.Reviews=google.Reviews.astype(int)
google.Reviews.dtype

dtype('int64')

In [112]:
#Show Updated Data Profile
Data_dict = pd.DataFrame(google.dtypes)
Data_dict = Data_dict.rename(columns={0:'Data_Type'})
Data_dict['Missing_Value'] = google.isnull().sum()
Data_dict['Unique_Value'] = google.nunique()
Data_dict['Count'] = google.count()
Data_dict

,Data_Type,Missing_Value,Unique_Value,Count
App,object,0,9660,10841
Category,object,1,33,10840
Rating,object,1474,40,9367
Reviews,int64,0,6001,10841
Size,object,0,461,10841
Installs,object,0,21,10841
Type,object,1,2,10840
Price,object,0,92,10841
Content_Rating,object,0,6,10841
Genres,object,1,119,10840


  **Rating** data type is change to object, we need to convert back to float
  

In [113]:
#Convert Rating to float
google = google.astype({'Rating':float})
google.Rating.dtypes

dtype('float64')

* **Convert Size data type to numeric**

In [114]:
#Check non numeric values in Size
google.Size[pd.to_numeric(google.Size, errors='coerce').isna()].value_counts()

Varies with device    1695
11M                    198
12M                    196
14M                    194
13M                    191
15M                    184
17M                    160
19M                    154
16M                    149
26M                    149
25M                    143
20M                    139
21M                    138
10M                    136
24M                    136
18M                    133
23M                    117
22M                    114
29M                    103
27M                     97
28M                     95
30M                     84
33M                     79
3.3M                    77
37M                     76
35M                     72
31M                     70
2.9M                    69
2.3M                    68
2.5M                    68
                      ... 
982k                     1
554k                     1
608k                     1
175k                     1
237k                     1
157k                     1
2

In [0]:
#Convert Size to numeric : 
#If last char ='k' then 0 to n-1 char multiply by 1024,  
#If last char ='M' then 0 to n-1 char multiply by 1024*1024,
#Else as it is,
google['Size'] = google['Size'].apply(lambda x : float(x[0:-1])*1024 if x[-1:]=='k' else (float(x[0:-1])*1024*1024 if x[-1:]=='M' else x))

In [116]:
#Check non numeric values in Size
google.Size[pd.to_numeric(google.Size, errors='coerce').isna()].value_counts()

Varies with device    1695
Name: Size, dtype: int64

  There are still non numeric values (**"Varies with device"**) in Size, but has no meaning. So we can just cast Size value to numeric

In [117]:
#Convert Reviews to Numeric
google['Size']=pd.to_numeric(google.Size, errors='coerce')
google.Size.dtype

dtype('float64')

* **Convert Installs data type to numeric**

In [118]:
#Check non numeric values in Installs
google.Installs[pd.to_numeric(google.Installs, errors='coerce').isna()].value_counts()

1,000,000+        1579
10,000,000+       1252
100,000+          1169
10,000+           1054
1,000+             908
5,000,000+         752
100+               719
500,000+           539
50,000+            479
5,000+             477
100,000,000+       409
10+                386
500+               330
50,000,000+        289
50+                205
5+                  82
500,000,000+        72
1+                  67
1,000,000,000+      58
0+                  14
Name: Installs, dtype: int64

In [0]:
#Remove '+' and ',' sign in Installs
google['Installs']=google['Installs'].apply(lambda x : x.replace('+','').replace(',',''))

In [120]:
#Check non numeric values in Installs again
google.Installs[pd.to_numeric(google.Installs, errors='coerce').isna()].value_counts()

Series([], Name: Installs, dtype: int64)

In [121]:
#Convert Install to numeric
google=google.astype({'Installs':int})
google.Installs.dtypes

dtype('int64')

* **Convert Price data type to numeric**

In [122]:
google.Price[pd.to_numeric(google.Price, errors='coerce').isna()].value_counts()

$0.99      148
$2.99      129
$1.99       73
$4.99       72
$3.99       63
$1.49       46
$5.99       30
$2.49       26
$9.99       21
$6.99       13
$399.99     12
$14.99      11
$4.49        9
$24.99       7
$7.99        7
$3.49        7
$29.99       7
$19.99       6
$5.49        6
$12.99       5
$6.49        5
$8.99        5
$11.99       5
$16.99       3
$1.00        3
$2.00        3
$10.00       3
$3.95        2
$10.99       2
$79.99       2
          ... 
$19.40       1
$1.96        1
$1.97        1
$28.99       1
$15.46       1
$3.02        1
$74.99       1
$4.84        1
$3.90        1
$4.85        1
$1.26        1
$109.99      1
$1.29        1
$3.61        1
$19.90       1
$5.00        1
$14.00       1
$89.99       1
$25.99       1
$400.00      1
$394.99      1
$4.59        1
$2.95        1
$3.04        1
$1.50        1
$2.50        1
$30.99       1
$379.99      1
$3.28        1
$37.99       1
Name: Price, Length: 91, dtype: int64

In [0]:
#Remove $ sign
google.Price=google.Price.apply(lambda x : x.replace('$',''))

In [124]:
#Check non numeric values in Price
google.Price[pd.to_numeric(google.Price,errors='coerce').isna()].value_counts()

Series([], Name: Price, dtype: int64)

In [125]:
#Convert Price to numeric
google=google.astype({'Price':float})
google.Price.dtypes

dtype('float64')

## **Handling Datetime Column Data Type**

 ### Last_Updated suppose to be Datetime

In [126]:
#Check Last_Updated value and format
google.Last_Updated.value_counts()

August 3, 2018       326
August 2, 2018       304
July 31, 2018        294
August 1, 2018       285
July 30, 2018        211
July 25, 2018        164
July 26, 2018        161
August 6, 2018       158
July 27, 2018        151
July 24, 2018        148
July 23, 2018        127
July 16, 2018        126
July 19, 2018        126
July 18, 2018        123
July 11, 2018        106
August 4, 2018       105
July 12, 2018        103
July 5, 2018          93
July 17, 2018         92
July 3, 2018          90
July 9, 2018          89
July 20, 2018         88
July 13, 2018         81
May 24, 2018          69
June 27, 2018         63
July 6, 2018          63
June 26, 2018         60
May 25, 2018          56
June 25, 2018         56
June 13, 2018         54
                    ... 
March 8, 2016          1
May 21, 2010           1
February 18, 2015      1
August 24, 2012        1
May 14, 2017           1
April 18, 2011         1
November 10, 2014      1
June 16, 2015          1
June 14, 2016          1


In [0]:
#Convert Last_Updatde to Datetime
from datetime import datetime
google['Last_Updated'] = google['Last_Updated'].apply(lambda x:  datetime.strptime(x, '%B %d, %Y'))

In [128]:
#Check Last_Updated data type
google.Last_Updated.dtypes

dtype('<M8[ns]')

In [129]:
#Show Data Profile
Data_dict = pd.DataFrame(google.dtypes)
Data_dict = Data_dict.rename(columns={0:'Data_Type'})
Data_dict['Missing_Value'] = google.isnull().sum()
Data_dict['Unique_Value'] = google.nunique()
Data_dict['Count'] = google.count()
Data_dict

,Data_Type,Missing_Value,Unique_Value,Count
App,object,0,9660,10841
Category,object,1,33,10840
Rating,float64,1474,39,9367
Reviews,int64,0,6001,10841
Size,float64,1695,459,9146
Installs,int64,0,20,10841
Type,object,1,2,10840
Price,float64,0,92,10841
Content_Rating,object,0,6,10841
Genres,object,1,119,10840


In [130]:
#Check Content
google.head(3)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Current_Ver,Android_Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19922944.0,10000,Free,0.0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14680064.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up
2,"U Launcher Lite â FREE Live Cool Themes, Hid...",ART_AND_DESIGN,4.7,87510,9122611.2,5000000,Free,0.0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up


## **Dataset is ready...**
<br>**We keep NA values as it is, without drop or impute**

# **HOMEWORK SOLUTION**

In [131]:
#Answer No 1
google.shape

(10841, 13)

In [132]:
#Answer No 2
google.Category.nunique()

33

In [133]:
#Answer No 3
google.Genres.nunique()

119

In [158]:
#Match Apps and Category based on provided dataset! 

#(Hint: Use App and Category columns)
#Answer No 7
AppNames=["File Commander - File Manager/Explorer",
"Pinterest",
"Subway Surfers",
"Flipboard: News For Our Time",
"WhatsApp Messenger",
"Dropbox",
"DU Battery Saver - Battery Charger & Battery Life",
"FIFA Soccer"]
df_temp=pd.DataFrame()
for AppName in AppNames:
  print(AppName + ':' + google[google.App==AppName].Category)


10253    File Commander - File Manager/Explorer:BUSINESS
Name: Category, dtype: object
2552    Pinterest:SOCIAL
2612    Pinterest:SOCIAL
4088    Pinterest:SOCIAL
Name: Category, dtype: object
1654    Subway Surfers:GAME
1700    Subway Surfers:GAME
1750    Subway Surfers:GAME
1872    Subway Surfers:GAME
1917    Subway Surfers:GAME
3896    Subway Surfers:GAME
Name: Category, dtype: object
3755    Flipboard: News For Our Time:NEWS_AND_MAGAZINES
3767    Flipboard: News For Our Time:NEWS_AND_MAGAZINES
3819    Flipboard: News For Our Time:NEWS_AND_MAGAZINES
Name: Category, dtype: object
336     WhatsApp Messenger:COMMUNICATION
381     WhatsApp Messenger:COMMUNICATION
3904    WhatsApp Messenger:COMMUNICATION
Name: Category, dtype: object
3473    Dropbox:PRODUCTIVITY
3569    Dropbox:PRODUCTIVITY
4048    Dropbox:PRODUCTIVITY
Name: Category, dtype: object
8896    DU Battery Saver - Battery Charger & Battery L...
Name: Category, dtype: object
8445    FIFA Soccer:SPORTS
Name: Category, dtype: obje

In [135]:
#Answer 8
# The following applications were installed by more than 1 billion users, EXCEPT ...
google[google.Installs >= 1000000000].groupby('App').App.nunique()

App
Facebook                                      1
Gmail                                         1
Google                                        1
Google Chrome: Fast & Secure                  1
Google Drive                                  1
Google News                                   1
Google Photos                                 1
Google Play Books                             1
Google Play Games                             1
Google Play Movies & TV                       1
Google Street View                            1
Google+                                       1
Hangouts                                      1
Instagram                                     1
Maps - Navigate & Explore                     1
Messenger â Text and Video Chat for Free    1
Skype - free IM & video calls                 1
Subway Surfers                                1
WhatsApp Messenger                            1
YouTube                                       1
Name: App, dtype: int64

In [136]:
google[google.App=='Facebook']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Current_Ver,Android_Ver
2544,Facebook,SOCIAL,4.1,78158306,NaN,1000000000,Free,0.0,Teen,Social,2018-08-03,Varies with device,Varies with device
3943,Facebook,SOCIAL,4.1,78128208,NaN,1000000000,Free,0.0,Teen,Social,2018-08-03,Varies with device,Varies with device


In [137]:
#Answer 9
#Arrange top COMMUNICATION category apps from the highest number of reviews to the lowest! 
#(Hint: Convert the data type to integer first. It is similar with code that was asked in question 6. 
google[google.Category=='COMMUNICATION'].loc[:,['App','Reviews']].drop_duplicates(subset='App', inplace=False).sort_values(by=['Reviews'], ascending=False)

,App,Reviews
336,WhatsApp Messenger,69119316
335,Messenger â Text and Video Chat for Free,56642847
378,UC Browser - Fast Download Private & Secure,17712922
406,BBM - Free Calls & Messages,12842860
342,Viber Messenger,11334799
403,LINE: Free Calls & Messages,10790289
391,Skype - free IM & video calls,10484169
338,Google Chrome: Fast & Secure,9642995
449,"Truecaller: Caller ID, SMS spam blocking & Dialer",7820209
365,WeChat,5387333


In [138]:
#Answer 10
#Arrange top 3 game category apps based on number of installed and rating!
google[google.Category=='GAME'].loc[:,['App','Installs','Rating']].drop_duplicates(subset='App', inplace=False).sort_values(by=['Installs','Rating'], ascending=False).head(3)

,App,Installs,Rating
1654,Subway Surfers,1000000000,4.5
1722,My Talking Tom,500000000,4.5
1655,Candy Crush Saga,500000000,4.4
